In [ ]:
import numpy as np
from numpy.fft import fft
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from qutip import *
from qutip import Bloch
import collections
import random
import time
import threading

## 想定するqubit系

以下のように, 2つの非調和振動子Q0, Q1がXX結合したハミルトニアンを仮定する:
$$
\begin{align}
\hat{H}
&= \hat{H}_0 + \hat{H}_1 + \hat{H}_{\rm int} + \hat{H}_{\rm d0} + \hat{H}_{\rm d1}\\
\hat{H}_i / \hbar
&= \omega_{{\rm q}i}\hat{a}_i^\dagger\hat{a}_i + \frac{\alpha_i}{2}\hat{a}_i^\dagger\hat{a}_i^\dagger\hat{a}_i\hat{a}_i
= 2\pi f_{{\rm q}i} \hat{a}_i^\dagger\hat{a}_i + \frac{2\pi f_{{\rm anharm}i}}{2}\hat{a}_i^\dagger\hat{a}_i^\dagger\hat{a}_i\hat{a}_i
\quad (i=0,1)\\
\hat{H}_{{\rm d}i} / \hbar
&= \varOmega_i\cos(\omega_{{\rm d}i}t + \phi_i)(\hat{a}_i^\dagger + \hat{a}_i)
= 2\pi f_{{\rm rabi}i}\cos(2\pi f_{{\rm d}i}t + \phi_i)(\hat{a}_i^\dagger + \hat{a}_i) \quad (i=0,1)\\
\hat{H}_{\rm int} / \hbar
& = g(\hat{a}_0\hat{a}_1^\dagger + {\rm H.c.}) = 2\pi f_g(\hat{a}_0\hat{a}_1^\dagger + {\rm H.c.}).
\end{align}
$$
このハミルトニアンの下3順位までを考慮する. 

緩和は縦緩和と純位相緩和を仮定する. 
状態の時間発展は以下のマスター方程式で表される: 
$$
\frac{\mathrm{d}\hat{\rho}}{\mathrm{d}t}
=
-\frac{\mathrm{i}}{\hbar}
[\hat{H}/\hbar,\hat{\rho}]
+\frac{1}{2}\sum_j
(2\hat{L}_j\hat{\rho}\hat{L}_j^\dagger - \hat{L}_j^\dagger\hat{L}_j\hat{\rho}-\hat{\rho}\hat{L}_j^\dagger\hat{L}_j)
$$
ここで$\hat{L}_{{\rm l}i}=\sqrt{\gamma_i}\hat{a}_i\ (i=0,1)$はレート$\gamma_i$の縦緩和, $\hat{L}_{{\rm p}i}=\sqrt{2\gamma_{{\rm p}i}}\hat{a}_i^\dagger\hat{a}_i\ (i=0,1)$はレート$\gamma_{{\rm p}i}$の純位相緩和に対応するリンドブラッド演算子である.

In [ ]:
#qubitパラメータ
f_q = [7500, 8300]
f_anharm = [-380, -400]
f_g = -10


#演算子の定義
N = 3 # 非調和3準位系を考える

a0 = tensor(destroy(N), qeye(N)) # Q1の消滅演算子
a1 = tensor(qeye(N), destroy(N)) # Q2の消滅演算子
a = [a0, a1]

P0_0 = tensor(basis(N,0) * basis(N,0).dag(), qeye(N)) # |0><0|
P1_0 = tensor(basis(N,1) * basis(N,1).dag(), qeye(N)) # |1><1|
Z_0 = P0_0 - P1_0 #sigma_z
X_0 = tensor(basis(N,0) * basis(N,1).dag() + basis(N,1) * basis(N,0).dag(), qeye(N)) #sigma_x
Y_0 = tensor(1j * basis(N,0) * basis(N,1).dag() - 1j * basis(N,1) * basis(N,0).dag(), qeye(N)) #sigma_y

P0_1 = tensor(qeye(N), basis(N,0) * basis(N,0).dag()) # |0><0|
P1_1 = tensor(qeye(N), basis(N,1) * basis(N,1).dag()) # |1><1|
Z_1 = P0_1 - P1_1 #sigma_z
X_1 = tensor(qeye(N), basis(N,0) * basis(N,1).dag() + basis(N,1) * basis(N,0).dag()) #sigma_x
Y_1 = tensor(qeye(N), 1j * basis(N,0) * basis(N,1).dag() - 1j * basis(N,1) * basis(N,0).dag()) #sigma_y

P0 = [P0_0, P0_1]
P1 = [P1_0, P1_1]
Z = [Z_0, Z_1]
X = [X_0, X_1]
Y = [Y_0, Y_1]


"""ハミルトニアン"""

# qubitの自由ハミルトニアン
H_q = []
for i in range(2):
    H_qi = 2*np.pi * f_q[i] * a[i].dag() * a[i] + (2*np.pi*f_anharm[i] / 2) * a[i].dag() * a[i].dag() * a[i] * a[i]
    H_q.append(H_qi)

# 駆動ハミルトニアンの時間無依存部分
H_d = []
for i in range(2):
    H_di = 2*np.pi * (a[i].dag() + a[i])
    H_d.append(H_di)

# 相互作用ハミルトニアン
H_int = 2*np.pi * f_g *(a[0] * a[1].dag() + a[0].dag() * a[1])

t_list = np.linspace(0, 0.2, 10001) # 時間リスト

# 全体のハミルトニアン
def H_tot(f_rabi, f_d, phi, t_list):
    H_tot = [H_q[0] + H_q[1] + H_int, 
             [H_d[0], np.real(f_rabi[0] * np.exp(1j * (2*np.pi * f_d[0] * t_list + phi[0])))], 
             [H_d[1], np.real(f_rabi[1] * np.exp(1j * (2*np.pi * f_d[1] * t_list + phi[1])))]
            ] 
    return H_tot


#リンドブラッド演算子
gamma = [0.1, 0.1]
L_l = []
for i in range(2):
    L_li = np.sqrt(gamma[i]) * a[i] # 縦緩和
    L_l.append(L_li)

gamma_p = [0.3, 0.3]
L_p = []
for i in range(2):
    L_pi = np.sqrt(2*gamma_p[i]) * a[i].dag() * a[i] # 純位相緩和
    L_p.append(L_pi)


# 初期状態
ini_state = tensor(basis(N,0), basis(N,0))   #　|0>|0>

# 密度行列から対角成分を取り出す
def diag_values(psi):
    array = []
    for i in range(len(psi[0][0])):
        array.append((psi[i][0][i]).real)
    return array

In [ ]:
# グラフのx軸の単位の設定
#sett = 1 # 1 ns
sett = 10 # 10ns
#sett = 5

had2 = 2**(-.5)*Qobj([[1,1,0],[1,-1,0],[0,0,0]])+Qobj([[0,0,0],[0,0,0],[0,0,1]])

def z_basis2(state):
    diag_n = len(state[0][0])
    n = int(diag_n**(.5))
    output = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            z = (state*tensor(had2,qeye(3)))[0][0][i*n+j]
            output[i][j] = abs(abs(z)*np.cos(np.angle(z)))
    return output

def z_basis(state):
    diag_n = len(state[0][0])
    n = int(diag_n**(.5))
    output = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            z = state[i*n+j][0][i*n+j]
            output[i][j] = abs(abs(z)*np.sin(np.angle(z)))
    return output

def x_basis(state):
    diag_n = len(state[0][0])
    output = np.zeros(diag_n)
    for i in range(diag_n):
        z = state[i][0][i]
        output[i] = abs(abs(z)*np.cos(np.angle(z)))
        #if output[i] < 10**(-8): output[i] = 0
    return output

#Initialize
state = ket2dm(ini_state)

In [ ]:
%matplotlib widget

qstates = []
count = 0

fig = plt.figure(figsize=(10,8), facecolor='#DEDEDE', tight_layout=True)

# define and adjust figure
gs = fig.add_gridspec(2,2)  
ax3 = fig.add_subplot(gs[0:1, 1:2], projection='3d')  
ax1 = fig.add_subplot(gs[0:1,0:1], facecolor="white") 
ax2 = fig.add_subplot(gs[1:2,0:1], facecolor="white") 
ax4 = fig.add_subplot(gs[1:2, 1:2], facecolor="white") 

plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True

# 量子状態の初期化
psi = (ini_state.dag()[0][0]).real
psi_upd = []

psi_z = (ini_state.dag()[0][0]).real
psi_z_upd = []

# X visualization

# 3D ヒストグラム作成
hist = psi
xedges = range(3)
yedges = range(3)
xpos, ypos = np.meshgrid(xedges , yedges, indexing="ij")
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = 0

dx = dy = 0.5 * np.ones_like(zpos)
dz = np.array(hist)

# 3D ヒストグラム カラー指定
cmap = cm.get_cmap('jet') 
max_height = 1.0
min_height = 0.0
rgba = [cmap(k) for k in dz] 

# 3D ヒストグラム プロット
ax3.bar3d(xpos, ypos, zpos, dx, dy, dz,color=rgba) 
plt.xticks(xpos, ["0","0","0","1","1","1","2","2","2"])
plt.yticks(ypos, ["0","1","2","0","1","2","0","1","2"])

# Z visualization
y = [0,0,0,-1,-1,-1,-2,-2,-2]
x = [0,1,2,0,1,2,0,1,2]
#ax4.hist2d(x,y,bins=3, density=None, weights=z_basis(state))
#ax4.axis("off")
#ax4.colorbar()        
cmap2 = cm.viridis
norm = Normalize(vmin=0, vmax=1)
aximg = ax4.imshow(z_basis2(state),norm=norm,cmap=cmap2)
ax4.set_xticks([0,1,2])
ax4.set_yticks([0,1,2])
#ax4.axis("off")
fig.colorbar(aximg, ax=ax4)

def my_function(i):
    # input control
    global qstates, count, qsave
    
    ctrl_input_q0_i.popleft()
    ctrl_input_q0_q.popleft()
    
    ctrl_input_q1_i.popleft()
    ctrl_input_q1_q.popleft()
    
    if len(qstates)>=1:
        ctrl_input_q0_i.append(qstates[0][0])
        ctrl_input_q0_q.append(qstates[0][1])
        
        ctrl_input_q1_i.append(qstates[0][2])
        ctrl_input_q1_q.append(qstates[0][3])
        
        '''q0_state0.append(qstates[0][2])
        q0_state1.append(qstates[0][3])
        q0_state2.append(qstates[0][4])'''
        
        #qsave = (qstates[0][2], qstates[0][3], qstates[0][4])
        qstates = qstates[1:]
    else:
        ctrl_input_q0_i.append(0)
        ctrl_input_q0_q.append(0)
        
        ctrl_input_q1_i.append(0)
        ctrl_input_q1_q.append(0)
    
   
    count += 1
    alist = np.arange(0+count,100+count,1)
    time.sleep(0.015)
        
    # clear axis
    ax1.cla()
    ax2.cla()
    
    ax1.set_title("Input Control Q0")
    ax2.set_title("Input Control Q1")
    ax3.set_title("X basis DM")
    ax4.set_title("Z basis DM")
 
    # plot input1    
    ax1.set_ylim(-1,1)
    ax1.plot(alist/(10*sett), ctrl_input_q0_i,color="blue", label="I")
    ax1.scatter(alist[-1]/(10*sett), ctrl_input_q0_i[-1],color="blue")
    
    ax1.plot(alist/(10*sett), ctrl_input_q0_q,color="red", label="Q")
    ax1.scatter(alist[-1]/(10*sett), ctrl_input_q0_q[-1],color="red")
    
    if sett == 1:        
        ax1.set_xlabel("time[ns]", {"fontsize":15})
        ax2.set_xlabel("time[ns]", {"fontsize":15})
    else:        
        ax1.set_xlabel("time["+str(sett)+"*ns]", {"fontsize":15})
        ax2.set_xlabel("time["+str(sett)+"*ns]", {"fontsize":15})
        
    ax1.set_ylabel("signal[amp]", {"fontsize":15})
    
    ax1.legend(loc=(0.05, 0.05))
    
    # plot input2
    #ax2.set_ylim(-1,1)
    ax2.set_ylim(-1,1)
    ax2.plot(alist/(10*sett), ctrl_input_q1_i,color="blue", label="I")
    ax2.scatter(alist[-1]/(10*sett), ctrl_input_q1_i[-1],color="blue")
    
    ax2.plot(alist/(10*sett), ctrl_input_q1_q,color="red", label="Q")
    ax2.scatter(alist[-1]/(10*sett), ctrl_input_q1_q[-1],color="red")
    
    ax2.set_ylabel("signal[amp]", {"fontsize":15})
    
    ax2.legend(loc=(0.05, 0.05))
    
    global atime, psi_upd
    if len(psi_upd)>0:
        ax3.clear()
        dz = psi_upd[0]
        cmap = cm.get_cmap('jet')
        
        rgba = [cmap(k) for k in dz] 
        ax3.bar3d(xpos, ypos, zpos, dx, dy, dz,color=rgba) 
        ax3.set_zlim(0,1.0)
        plt.xticks(xpos, ["0","0","0","1","1","1","2","2","2"])
        plt.yticks(ypos, ["0","1","2","0","1","2","0","1","2"])
        
        ax4.clear()                
        #ax4.hist2d(x,y,bins=3, weights=z_basis(state),cmap=cm.get_cmap('jet'))
        #ax4.axis("off")
        #aximg = ax4.imshow(x,y)
        #fig.colorbar(aximg, ax=ax4, shrink=0.62)
        
        cmap2 = cm.viridis
        norm = Normalize(vmin=0, vmax=1)
        aximg = ax4.imshow(z_basis2(state),norm=norm,cmap=cmap2)
        #ax4.axis("off")
        #fig.colorbar(aximg, ax=ax4)
        
        psi_upd = psi_upd[1:]
        #psi_z_upd = psi_z_upd[1:]
        #time.sleep(0.4)

# start collections with zeros
ctrl_input_q0_i = collections.deque(np.zeros(100))
ctrl_input_q0_q = collections.deque(np.zeros(100))
ctrl_input_q1_i = collections.deque(np.zeros(100))
ctrl_input_q1_q = collections.deque(np.zeros(100))

# animate

ani = FuncAnimation(fig, my_function, interval=100)
plt.show()

In [ ]:
#Drive setting
f_rabi = [20, 20] # 振幅
f_d = [0, 0] # 周波数
f_d[0] = f_q[0]
f_d[1] = f_q[1]
phi = [0, 0] # 位相

#dt = 0.00004
#duration = 0.04
#t_list = np.linspace(0, 0 + duration, round(duration/dt)+1)

#drive_t_list = np.array(t_list) * np.array([[0 + 0j], [0 + 0j]]) # 初期化

In [ ]:
# Q0-> X Gate 1/2
f_rabi = [20, 20] # 振幅
A = f_rabi[0]
f_carrier = f_d[0]
driven_qubit_num = 0

I = [1,0]
Q = [0,0]

duration = 0.027*.5 # 単位: us
dt = 0.00004 # 単位: us
t_list = np.linspace(0, 0 + duration, round(duration/dt)+1)
drive_t_list = np.array(t_list) * np.array([[0 + 0j], [0 + 0j]]) # 初期化

waveform_list = np.array([A * np.exp(1j * phi[0]) for i in t_list])
drive_t_list[driven_qubit_num] = waveform_list * np.exp(1j * 2*np.pi * f_carrier * t_list)    
H_drive = [H_q[0] + H_q[1] + H_int, 
                   [H_d[0], np.real(drive_t_list[0])], 
                   [H_d[1], np.real(drive_t_list[1])]
                  ] 

# round(duration/dt)の標本数のうち50個をプロットする
samples = round(duration/dt)

#ts_f = duration

In [ ]:
# Q0-> X Gate
f_rabi = [20, 20] # 振幅
A = f_rabi[0]
f_carrier = f_d[0]
driven_qubit_num = 0

I = [1,0]
Q = [0,0]

duration = 0.027 # 単位: us
dt = 0.00004 # 単位: us
t_list = np.linspace(0, 0 + duration, round(duration/dt)+1)
drive_t_list = np.array(t_list) * np.array([[0 + 0j], [0 + 0j]]) # 初期化

waveform_list = np.array([A * np.exp(1j * phi[0]) for i in t_list])
drive_t_list[driven_qubit_num] = waveform_list * np.exp(1j * 2*np.pi * f_carrier * t_list)    
H_drive = [H_q[0] + H_q[1] + H_int, 
                   [H_d[0], np.real(drive_t_list[0])], 
                   [H_d[1], np.real(drive_t_list[1])]
                  ] 

# round(duration/dt)の標本数のうち50個をプロットする
samples = round(duration/dt)

#ts_f = duration

In [ ]:
# Q0-> P Gate
H_drive = [H_q[0] + H_q[1] + H_int] 

theta=np.pi
#RZgt = Qobj([[np.e**(-1j*theta/2),0,0],[0,np.e**(1j*theta/2),0],[0,0,1]])
Pgt = Qobj([[1,0,0],[0,np.e**(1j*theta),0],[0,0,1]])
#Tgt = Qobj([[1,0,0],[0,np.e**(1j*theta/4),0],[0,0,1]])
t_list = np.linspace(0, 0 + dt, 2)

#result = mesolve(H_drive, state*tensor(Pgt,qeye(3)), t_list, [])
state=state*tensor(Pgt,qeye(3))
#state = result.states[-1]

psi_upd.append(x_basis(state))
#qstates.append([f_rabi[0]*(I[0]*np.sin(2*np.pi*i*f_d[0]*(duration/(samples*10)) + phi[0])),f_rabi[0]*(Q[0]*np.cos(2*np.pi*i*f_d[0]*(duration/(samples*10)) + phi[0])), f_rabi[1]*(I[1]*np.sin(2*np.pi*i*f_d[1]*(duration/(samples*10)) + phi[1])),f_rabi[1]*(Q[1]*np.cos(2*np.pi*i*f_d[1]*(duration/(samples*10)) + phi[1]))]) 
#qstates.append([(I[0]*np.sin(2*np.pi*i*f_d[0]*(duration/(samples*10)) + phi[0])),(Q[0]*np.cos(2*np.pi*i*f_d[0]*(duration/(samples*10)) + phi[0])), (I[1]*np.sin(2*np.pi*i*f_d[1]*(duration/(samples*10)) + phi[1])),(Q[1]*np.cos(2*np.pi*i*f_d[1]*(duration/(samples*10)) + phi[1]))]) 

In [ ]:
# Q1-> X Gate
A = 20
f_carrier = f_d[1]
driven_qubit_num = 1

I = [0,1]
Q = [0,0]


duration = 0.027 # 単位: us
dt = 0.00004 # 単位: us
t_list = np.linspace(0, 0 + duration, round(duration/dt)+1)
drive_t_list = np.array(t_list) * np.array([[0 + 0j], [0 + 0j]]) # 初期化

waveform_list = np.array([A * np.exp(1j * phi[0]) for i in t_list])
drive_t_list[driven_qubit_num] = waveform_list * np.exp(1j * 2*np.pi * f_carrier * t_list)    
H_drive = [H_q[0] + H_q[1] + H_int, 
                   [H_d[0], np.real(drive_t_list[0])], 
                   [H_d[1], np.real(drive_t_list[1])]
                  ] 

# round(duration/dt)の標本数のうち50個をプロットする
samples = round(duration/dt)

#ts_f = duration

In [ ]:
#Initialize
state = ket2dm(ini_state)

In [ ]:
# Drive
result = mesolve(H_drive, state, t_list, [])
#k = int(len(t_list)/samples)
for i in range(0,samples+1,sett):
    psi_upd.append(x_basis(result.states[i]))
    #qstates.append([f_rabi[0]*(I[0]*np.sin(2*np.pi*i*f_d[0]*(duration/(samples*10)) + phi[0])),f_rabi[0]*(Q[0]*np.cos(2*np.pi*i*f_d[0]*(duration/(samples*10)) + phi[0])), f_rabi[1]*(I[1]*np.sin(2*np.pi*i*f_d[1]*(duration/(samples*10)) + phi[1])),f_rabi[1]*(Q[1]*np.cos(2*np.pi*i*f_d[1]*(duration/(samples*10)) + phi[1]))]) 
    qstates.append([(I[0]*np.sin(2*np.pi*i*f_d[0]*(duration/(samples*10)) + phi[0])),(Q[0]*np.cos(2*np.pi*i*f_d[0]*(duration/(samples*10)) + phi[0])), (I[1]*np.sin(2*np.pi*i*f_d[1]*(duration/(samples*10)) + phi[1])),(Q[1]*np.cos(2*np.pi*i*f_d[1]*(duration/(samples*10)) + phi[1]))]) 
state = result.states[-1]